<a href="https://colab.research.google.com/github/SezerBugday/photogrammetry_meshroom_colab_flask/blob/main/Bitirme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 main.py

In [49]:
from pyngrok import ngrok

import os  
import shutil  


from flask import Flask, flash, request, redirect, url_for,render_template
from werkzeug.utils import secure_filename

port_no = 5000

import sqlite3 as sql


con = sql.connect("/content/sezerdb.db", check_same_thread=False)
cursor = con.cursor()

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import sys




In [55]:
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

TEMPLATE = "/content/photogrammetry_meshroom_colab_flask/templates"
app = Flask(__name__, template_folder=TEMPLATE)
app._static_folder = '/content/static/'

#bootsrapt = Bootstrap(app)
islem_durumu=False;

app.config['UPLOAD_FOLDER'] = "/content/kirli"

ngrok.set_auth_token("2DJyoS7QMsne6ZsjPBWlik5uBBv_37hajTNHz6NFWXeczXf7A")
public_url = ngrok.connect(port_no).public_url

print(f"To acces the Gloable link please click {public_url}")


@app.route("/")
def main_page():
    print("Anasayfaya donuldu")
    return render_template("HomePage.html")

@app.route("/uploadimages", methods=["GET", "POST"])
def upload_file():
    print("resimler secildi")
    if request.method == 'POST':
        files = request.files.getlist("image")
        for file in files:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], file.filename))
            print("Görseller kaydedildi")
            return redirect(url_for('Show_Process'))
    return render_template("ShowSelectedImages.html")
  
@app.route("/process")
def Show_Process():
  if(islem_durumu==False):
    return redirect(url_for('Process'))
  return render_template("processing.html")
  print("isleniyor")

  
@app.route("/sezer")
def Process():



  return render_template("showModel.html")
  return "çalisti"
  os.system("rembg p /content/kirli   /content/temiz")
  os.system("./meshroom/Meshroom-2019.1.0/meshroom_photogrammetry --input /content/temiz --output /content/modelFolder")
  os.system("obj2gltf -i /content/modelFolder/model.obj -o /content/static/model.glb")
  
@app.route("/SingUp", methods=["GET", "POST"])
def SingUp():
   if request.method == 'POST':
        name = request.form.get("name")
        surname = request.form.get("surname")
        email = request.form.get("email")
        password = request.form.get("password")
        print("before db")
        print(name)
        #cursor.execute("INSERT INTO Users VALUES(name,surname,password,email)")
        cursor.execute("insert into Users values(?,?,?,?)",(name,surname,email,password))
        con.commit()
        print("after db")
        # Gmail email sunucusuna bağlanıyoruz
        try:
            mail = smtplib.SMTP("smtp.gmail.com",587)
            mail.ehlo()
            mail.starttls()
            mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")

            mesaj = MIMEMultipart()
            mesaj["From"] = "3D AR Object Platform "           # Gönderen
            mesaj["Subject"] = "Verification Code"    # Konusu
            mesaj["To"] = email
      
       
            body = "Your Verification Code :  22 03 22 "

            body_text = MIMEText(body, "plain")  #
            mesaj.attach(body_text)

            mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
            print("Mail başarılı bir şekilde gönderildi.")
            mail.close()
            return render_template("SentVerificationCode.html")
        # Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
        except:
            print("Hata:", sys.exc_info()[0])
            
   return render_template("SingUp.html")

@app.route("/Login")
def Login():
  return render_template("LoginPage.html")

if __name__ == "__main__":
    app.run()

To acces the Gloable link please click http://4d5d-34-90-228-175.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:06] "GET / HTTP/1.1" 200 -


Anasayfaya donuldu


INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:10] "GET /uploadimages HTTP/1.1" 200 -


resimler secildi
resimler secildi


...

INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:17] "GET /process HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:18] "GET /sezer HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:18] "GET /node_modules/xterm/css/xterm.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:19] "GET /static/kup.glb HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:22] "GET /SingUp?submit=Sing+Up HTTP/1.1" 200 -


before db
sedfs
after db


INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:34] "POST /SingUp HTTP/1.1" 200 -


Mail başarılı bir şekilde gönderildi.


INFO:werkzeug:127.0.0.1 - - [21/Dec/2022 22:23:39] "GET /dene HTTP/1.1" 404 -


In [40]:


# Gmail email sunucusuna bağlanıyoruz
try:
    mail = smtplib.SMTP("smtp.gmail.com",587)
    mail.ehlo()
    mail.starttls()
    mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")

    mesaj = MIMEMultipart()
    mesaj["From"] = "3D AR Object Platform "           # Gönderen
    mesaj["Subject"] = "Verification Code"    # Konusu
    mesaj["To"] = "sezerbugday2@gmail.com "
    body = " Bu mesaj sezer tarafından python ile gönderilmiştir "

    body_text = MIMEText(body, "plain")  #
    mesaj.attach(body_text)

    mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
    print("Mail başarılı bir şekilde gönderildi.")
    mail.close()

# Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
except:
    print("Hata:", sys.exc_info()[0])

Mail başarılı bir şekilde gönderildi.


In [39]:
cursor.execute("Select * From Users")
veri = cursor.fetchall() 
print(veri)

[('sezer', 'bugday', '123', 'deneme'), ('1', '2', '3', '4'), ('1', '2', '3', '4'), ('555', '1', '2', '3'), ('888', '1', '2', '3'), ('sezer', 'dsd', 'sdfdsf', 'hfghffhg')]


In [21]:
name="1"
surname="2"
email="3"
password ="4"
cursor.execute("insert into Users values(?,?,?,?)",(name, surname, email,password))
con.commit()